<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">

## Declare Factory

In [ ]:
TMVA::Tools::Instance();

TFile* outputFile = TFile::Open( "TMVAOutputVI.root", "RECREATE" );
TFile* inputFile  = TFile::Open( "inputdata.root");

TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,
                                               "!V:!ROC:SilentFile:!Correlations:Silent:Color:!DrawProgressBar:AnalysisType=Classification" );

## Declare DataLoader

In [ ]:
TMVA::DataLoader *loader=new TMVA::DataLoader("dataset");

loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", 'F' );
loader->AddVariable( "var3", 'F' );
loader->AddVariable( "var4", 'F' );
loader->AddVariable( "var5 := var1*var2", 'F' );

## Setting up dataset

In [ ]:
TTree *tsignal     = (TTree*)inputFile->Get("Sig");
TTree *tbackground = (TTree*)inputFile->Get("Bkg");

TCut mycuts = ""; 
TCut mycutb = ""; 
   
loader->AddSignalTree     (tsignal, 1 );      //signal weight = 1
loader->AddBackgroundTree (tbackground, 1 );  //background weight = 1 

loader->PrepareTrainingAndTestTree( mycuts, mycutb, "nTrain_Signal=1000:nTrain_Background=1000:nTest_Signal=500:nTest_Background=500:SplitMode=Random:NormMode=NumEvents:!V" );

## Variable importance

In [ ]:
TH1F* importance = factory->EvaluateImportance(loader,TMVA::VIType::kAll,TMVA::Types::kBDT, "BDT","!V:NTrees=5:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

In [ ]:
%jsroot
TCanvas *canvas=new TCanvas("Importance");
importance->Draw();
canvas->Draw();

In [ ]:
outputFile->Close();
delete factory;
delete loader;
delete canvas;